In [1]:
using Revise
using BilevelTrajOpt

In [2]:
a = rand(4)
A = diagm(a)

4×4 Array{Float64,2}:
 0.535168  0.0       0.0       0.0     
 0.0       0.478549  0.0       0.0     
 0.0       0.0       0.673624  0.0     
 0.0       0.0       0.0       0.759276

In [167]:
# A = diagm(rand(4))

# test the solution
function f(x)
#     return x[1] * x[4] * (x[1] + x[2] + x[3]) + x[3]
#     return x'*A*x 
#     return 0.
    return sum(x)
end

function h(x)
#     return [40. - (x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2)]
    return [6. - x[1],x[1]*x[2],x[1]*.5+x[3]-4.]
#     return [x[1] + x[2] - 1., x[3] - 1., x[4]*x[1]]
end

function g(x)
#     return [25. - (x[1]*x[2]*x[3]*x[4])]
#     return vcat([25. - (x[1]*x[2]*x[3]*x[4])], 1. - x, x - 5.)
#     return vcat([1. - x[2]],-x)

    return vcat(-x[2:3],5.-x[4],10.-x[4],17.-x[4],5-x[1]*x[4])
#     return vcat(-x[2:3],5.-x[4])
    
#     return [0.]
#     return vcat([x[1] - .1], 5. - x[4])
end

x0 = zeros(4)
λ0 = zeros(length(h(x0)))
μ0 = zeros(length(g(x0)))
c0 = 1.

# x0 = x_sol
# λ0 = λ_sol
# μ0 = μ_sol
# c0 = c_sol

x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g,c0=c0)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g)

# x_sol_known = [1.000, 4.743, 3.821, 1.379]

display(x_sol)
display(x_sol_ip)
# display(x_sol_known)

10-element Array{Float64,1}:
  0.342591 
 -0.0158541
  0.167881 
  1.49031  
 -0.0184756
 -0.0136877
 -0.10952  
 -0.239813 
 -0.422224 
 -0.148356 

10-element Array{Float64,1}:
  0.394748 
 -0.0137292
  0.182168 
  1.5988   
 -0.0196827
 -0.0145925
 -0.116444 
 -0.253876 
 -0.444466 
 -0.156543 

10-element Array{Float64,1}:
  0.399684 
 -0.013508 
  0.183489 
  1.60872  
 -0.0197934
 -0.0146729
 -0.117056 
 -0.255136 
 -0.446459 
 -0.157281 

10-element Array{Float64,1}:
   6.0     
   0.189271
   1.0     
  10.5665  
  -1.01773 
   0.618295
  10.3556  
  -0.271554
 -18.2016  
  16.3698  

10-element Array{Float64,1}:
   6.0        
   6.98608e-14
   1.0        
  17.0        
  -2.17991    
   0.54237    
  12.0        
  14.6026     
 -18.2016     
  97.0        

4-element Array{Float64,1}:
  6.0        
  6.98608e-14
  1.0        
 17.0        

4-element Array{Float64,1}:
  6.0        
  2.16408e-24
  1.0        
 17.0        

Solve_Succeeded


In [90]:
μ_sol

4-element Array{Float64,1}:
    0.14712
   -4.65494
 -222.0    
 -227.0    

In [113]:
x = x_sol
display(f(x))
display(h(x))
display(g(x))

x = x_sol_ip
display(f(x))
display(h(x))
display(g(x))

17.389794243745335

3-element Array{Float64,1}:
  4.05533e-11
 -4.58422e-15
  5.29443e-12

4-element Array{Float64,1}:
  7.64037e-16
 -1.0        
 -5.38979    
 -0.389794   

16.999999990009997

3-element Array{Float64,1}:
  0.0        
 -1.24077e-24
  0.0        

4-element Array{Float64,1}:
  2.06795e-25
 -1.0        
 -5.0        
  9.99e-9    

In [ ]:
# test the gradient

A = diagm(rand(4))

function solve_prob(z)
    function f(x)
        return x'*A*x + z[1]
    end

    function h(x)
        return [x[1] + x[2] - 1., x[3] - z[2], x[4]*x[1]]
    end

    function g(x)
        return vcat([x[1] - z[4]], z[3] - x[4])
    end

    x0 = [1., 1., 1., 1.]
    λ0 = ones(length(h(x0)))
    μ0 = ones(length(g(x0)))

    x_min = 1.*ones(length(x0))
    x_max = 10.*ones(length(x0))

    x_sol, λ_sol, μ_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)
    
    x_sol
end

In [ ]:
z0 = [2.,-1.]

# autodiff 
J_auto = ForwardDiff.jacobian(solve_prob,z0)

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - solve_prob(z0))/ϵ
end

display(solve_prob(z0))

display(J_auto)
display(J_num)